In [2]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F
import lbl2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

In [3]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

22/10/04 19:24:12 WARN Utils: Your hostname, Aishwaryas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.6.81 instead (on interface en0)
22/10/04 19:24:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 19:24:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/04 19:24:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/04 19:24:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/04 19:24:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
consumer = pd.read_csv("/data/tables/tbl_consumer.csv", delimiter="|")

FileNotFoundError: [Errno 2] No such file or directory: '/data/tables/tbl_consumer.csv'

In [ ]:
consumer.head()

In [ ]:
merchants = spark.read.parquet("/Users/Kasturi/Documents/GitHub/generic-buy-now-pay-later-project-group-10-bnpl/data/tables/tbl_merchants.parquet")

In [ ]:
merchants_df = merchants.toPandas()

In [ ]:
merchants_df.head()

In [ ]:
# Separate tags from revenue and take rate
tags = merchants_df["tags"].str.split("\), ", expand=True)
tags = tags[0].str.split("\], ", expand=True)
# Remove symbols from tag and making everything lowercase
tags = tags[0].str.replace('[^\w\s]', '', regex = True)
tags = tags.str.lower()

In [ ]:
merchants_df['cleaned_tags'] = tags

In [ ]:
merchants_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(merchants_df['cleaned_tags'])

In [ ]:
categories_label = ["fashion", "furniture", "electronics", "beauty, health, personal and household", "toys, hobbies and DIY"]
len(categories_label)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=5,random_state=42)
LDA.fit(dtm)
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

In [ ]:
topic_results = LDA.transform(dtm)
merchants_df['store_type'] = topic_results.argmax(axis=1)
merchants_df.head(10)

In [ ]:
myDict = {0 : 'Furniture' , 1 : 'Toys and DIY', 2 : 'Beauty, Health, Personal and Household', 3 : 'Books, Stationary and Music', 4 : 'Electronics' }
 
merchants_df['category'] = merchants_df['store_type'].map(myDict)

In [ ]:
merchants_df.head()

In [ ]:
merchants_df.to_csv("../data/curated/tagged_merchants.csv")